In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

np.random.seed(42)

data = pd.read_csv('star_classification.csv', delimiter=",")

features = ['alpha','delta','u','g','r','i','z','redshift']

for col in features:
    mean = data[col].mean()
    std = data[col].std()
    data = data[(data[col] >= mean - 3*std) & (data[col] <= mean + 3*std)]

le = LabelEncoder()
data['class_encoded'] = le.fit_transform(data['class'])

X = data[features].values
y_encoded = data['class_encoded'].values
y = np.eye(3)[y_encoded]  # one-hot

combined = np.concatenate((X, y), axis=1)

def split_data(data, train_ratio=0.8):
    np.random.shuffle(data)
    train_size = int(len(data) * train_ratio)
    train_data = data[:train_size]
    test_data = data[train_size:]
    return train_data, test_data

train_data, test_data = split_data(combined)

train_df = pd.DataFrame(train_data, columns=features + ['c0','c1','c2'])
train_df['class'] = np.argmax(train_df[['c0','c1','c2']].values, axis=1)
min_count = train_df['class'].value_counts().min()
balanced_train_df = pd.concat([
    df.sample(min_count, random_state=42)
    for _, df in train_df.groupby('class')
])
train_data = balanced_train_df.drop(columns='class').values

X_train = train_data[:, :len(features)]
y_train = train_data[:, len(features):]
X_test = test_data[:, :len(features)]
y_test = test_data[:, len(features):]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train_lbl = np.argmax(y_train, axis=1)
y_test_lbl = np.argmax(y_test, axis=1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

np.random.seed(42)

def relu(x, derivative=False):
    if derivative:
        return (x > 0).astype(float)
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def initialize_parameters(layer_sizes):
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i])
        b = np.zeros((1, layer_sizes[i + 1]))
        weights.append(W)
        biases.append(b)
    return weights, biases

def forward(X, weights, biases):
    activations = [X]
    for W, b in zip(weights[:-1], biases[:-1]):
        A = relu(activations[-1] @ W + b)
        activations.append(A)
    A_out = softmax(activations[-1] @ weights[-1] + biases[-1])
    activations.append(A_out)
    return activations

def backward(activations, weights, y):
    deltas = [activations[-1] - y]
    for i in reversed(range(len(weights) - 1)):
        delta = deltas[0] @ weights[i + 1].T
        delta *= relu(activations[i + 1], derivative=True)
        deltas.insert(0, delta)

    grads_W, grads_b = [], []
    for i in range(len(weights)):
        grads_W.append(activations[i].T @ deltas[i] / len(y))
        grads_b.append(np.mean(deltas[i], axis=0, keepdims=True))
    return grads_W, grads_b

def cross_entropy(y_true, y_pred):
    eps = 1e-9
    return -np.mean(np.sum(y_true * np.log(y_pred + eps), axis=1))

def create_mini_batches(X, y, batch_size, shuffle=True): # Ważne dla momentum
    if shuffle:
        indices = np.random.permutation(len(X))
        X = X[indices]
        y = y[indices]

    for start in range(0, len(X), batch_size):
        end = start + batch_size
        yield X[start:end], y[start:end]

def train(X, y, hidden_layers, lr, epochs, batch_size,
          optimizer, momentum):
    layer_sizes = [X.shape[1]] + hidden_layers + [y.shape[1]]
    weights, biases = initialize_parameters(layer_sizes)
    loss_history = []

    vW = [np.zeros_like(w) for w in weights]
    vb = [np.zeros_like(b) for b in biases]
    mW = [np.zeros_like(w) for w in weights]
    mb = [np.zeros_like(b) for b in biases]
    sW = [np.zeros_like(w) for w in weights]
    sb = [np.zeros_like(b) for b in biases]
    eps = 1e-8

    for epoch in range(epochs):
        epoch_loss = 0
        batch_count = 0

        for X_batch, y_batch in create_mini_batches(X, y, batch_size):
            activations = forward(X_batch, weights, biases)
            grads_W, grads_b = backward(activations, weights, y_batch)

            for i in range(len(weights)):
                if optimizer == "sgd":
                    weights[i] -= lr * grads_W[i]
                    biases[i] -= lr * grads_b[i]

                elif optimizer == "momentum":
                    vW[i] = momentum * vW[i] - lr * grads_W[i]
                    vb[i] = momentum * vb[i] - lr * grads_b[i]
                    weights[i] += vW[i]
                    biases[i] += vb[i]

            batch_loss = cross_entropy(y_batch, activations[-1])
            epoch_loss += batch_loss
            batch_count += 1
        
        loss_history.append(epoch_loss / batch_count)
        
    return weights, biases, loss_history

def predict(X, weights, biases):
    A = X
    for W, b in zip(weights[:-1], biases[:-1]):
        A = relu(A @ W + b)
    A = softmax(A @ weights[-1] + biases[-1])
    return np.argmax(A, axis=1)

def calculate_classification_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return acc, prec, rec, f1

learning_rates = [0.001, 0.01, 0.05, 0.1]
epochs_list = [1500, 1200, 900, 500]
hidden_layer_configs = [
        [128, 64, 32, 16], [64, 32, 16, 8], [32, 16, 8, 4],
        [64, 32, 16], [32, 16, 8], [16, 8, 4],
        [64, 32], [32, 16], [16, 8], [8, 4],
        [64], [32], [16], [8]
]
optimizers = ["sgd", "momentum"]
momentum_values = [0.6, 0.7, 0.8, 0.9]
repeat = 5

base_params = {
    'hidden_layers': [32, 16, 8],
    'optimizer': 'sgd',
    'momentum': 0.9,
    'learning_rate': 0.1,
    'epochs': 500
}

In [ ]:
results = []

for r in range(1, repeat + 1):
    # 1. Iteracja po hidden_layers
    for hl in hidden_layer_configs:
        params = base_params.copy()
        params['hidden_layers'] = hl
        
        trained_weights, trained_biases, loss_history = train(
            X_train, y_train,
            hidden_layers=params['hidden_layers'],
            lr=params['learning_rate'],
            epochs=params['epochs'],
            batch_size=100, 
            optimizer=params['optimizer'],
            momentum=params['momentum']
        )
        final_loss = loss_history[-1]  # bierzemy wartość loss z ostatniej epok

        pred_train = predict(X_train, trained_weights, trained_biases)
        pred_test = predict(X_test, trained_weights, trained_biases)
        acc_train, prec_train, rec_train, f1_train = calculate_classification_metrics(y_train_lbl, pred_train)
        acc_test, prec_test, rec_test, f1_test = calculate_classification_metrics(y_test_lbl, pred_test)

        results.append({
            'hidden_layers': str(params['hidden_layers']),
            'optimizer': params['optimizer'],
            'momentum': params['momentum'],
            'learning_rate': params['learning_rate'],
            'epochs': params['epochs'],
            'repeat': r,
            'acc_train': acc_train,
            'precision_train': prec_train,
            'recall_train': rec_train,
            'f1_train': f1_train,
            'acc_test': acc_test,
            'precision_test': prec_test,
            'recall_test': rec_test,
            'f1_test': f1_test,
            'final_loss': final_loss
        })

    # 2. Iteracja po learning_rate
    for lr in learning_rates:
        params = base_params.copy()
        params['learning_rate'] = lr
        
        trained_weights, trained_biases, loss_history = train(
            X_train, y_train,
            hidden_layers=params['hidden_layers'],
            lr=params['learning_rate'],
            epochs=params['epochs'],
            batch_size=100, 
            optimizer=params['optimizer'],
            momentum=params['momentum']
        )
        final_loss = loss_history[-1]

        pred_train = predict(X_train, trained_weights, trained_biases)
        pred_test = predict(X_test, trained_weights, trained_biases)
        acc_train, prec_train, rec_train, f1_train = calculate_classification_metrics(y_train_lbl, pred_train)
        acc_test, prec_test, rec_test, f1_test = calculate_classification_metrics(y_test_lbl, pred_test)

        results.append({
            'hidden_layers': str(params['hidden_layers']),
            'optimizer': params['optimizer'],
            'momentum': params['momentum'],
            'learning_rate': params['learning_rate'],
            'epochs': params['epochs'],
            'repeat': r,
            'acc_train': acc_train,
            'precision_train': prec_train,
            'recall_train': rec_train,
            'f1_train': f1_train,
            'acc_test': acc_test,
            'precision_test': prec_test,
            'recall_test': rec_test,
            'f1_test': f1_test,
            'final_loss': final_loss
        })

    # 3. Iteracja po epochs
    for ep in epochs_list:
        params = base_params.copy()
        params['epochs'] = ep
        
        trained_weights, trained_biases, loss_history = train(
            X_train, y_train,
            hidden_layers=params['hidden_layers'],
            lr=params['learning_rate'],
            epochs=params['epochs'],
            batch_size=100, 
            optimizer=params['optimizer'],
            momentum=params['momentum']
        )
        final_loss = loss_history[-1]

        pred_train = predict(X_train, trained_weights, trained_biases)
        pred_test = predict(X_test, trained_weights, trained_biases)
        acc_train, prec_train, rec_train, f1_train = calculate_classification_metrics(y_train_lbl, pred_train)
        acc_test, prec_test, rec_test, f1_test = calculate_classification_metrics(y_test_lbl, pred_test)

        results.append({
            'hidden_layers': str(params['hidden_layers']),
            'optimizer': params['optimizer'],
            'momentum': params['momentum'],
            'learning_rate': params['learning_rate'],
            'epochs': params['epochs'],
            'repeat': r,
            'acc_train': acc_train,
            'precision_train': prec_train,
            'recall_train': rec_train,
            'f1_train': f1_train,
            'acc_test': acc_test,
            'precision_test': prec_test,
            'recall_test': rec_test,
            'f1_test': f1_test,
            'final_loss': final_loss
        })

    # 4. Iteracja po optimizer i momentum
    for opt in optimizers:
        moms = momentum_values if opt == "momentum" else [0.0]
        for mom in moms:
            params = base_params.copy()
            params['optimizer'] = opt
            params['momentum'] = mom
            
            trained_weights, trained_biases, loss_history = train(
                X_train, y_train,
                hidden_layers=params['hidden_layers'],
                lr=params['learning_rate'],
                epochs=params['epochs'],
                batch_size=100, 
                optimizer=params['optimizer'],
                momentum=params['momentum']
            )
            final_loss = loss_history[-1]

            pred_train = predict(X_train, trained_weights, trained_biases)
            pred_test = predict(X_test, trained_weights, trained_biases)
            acc_train, prec_train, rec_train, f1_train = calculate_classification_metrics(y_train_lbl, pred_train)
            acc_test, prec_test, rec_test, f1_test = calculate_classification_metrics(y_test_lbl, pred_test)

            results.append({
                'hidden_layers': str(params['hidden_layers']),
                'optimizer': params['optimizer'],
                'momentum': params['momentum'],
                'learning_rate': params['learning_rate'],
                'epochs': params['epochs'],
                'repeat': r,
                'acc_train': acc_train,
                'precision_train': prec_train,
                'recall_train': rec_train,
                'f1_train': f1_train,
                'acc_test': acc_test,
                'precision_test': prec_test,
                'recall_test': rec_test,
                'f1_test': f1_test,
                'final_loss': final_loss
            })

df_mlp = pd.DataFrame(results)
df_mlp.to_excel('mlp_sekwencyjne_nasze_dane.xlsx', index=False)

In [ ]:
# Zagregowane wyniki (średnia po powtórzeniach)
summary_mlp = df_mlp.groupby(['hidden_layers', 'optimizer', 'momentum', 'learning_rate', 'epochs']) \
    .agg({
        'acc_train': ['mean', 'min', 'max'],
        'precision_train': ['mean', 'min', 'max'],
        'recall_train': ['mean', 'min', 'max'],
        'f1_train': ['mean', 'min', 'max'],
        'acc_test': ['mean', 'min', 'max'],
        'precision_test': ['mean', 'min', 'max'],
        'recall_test': ['mean', 'min', 'max'],
        'f1_test': ['mean', 'min', 'max'],
        'final_loss': ['mean', 'min', 'max']
    }).reset_index()

summary_mlp.columns = ['_'.join(col).strip('_') for col in summary_mlp.columns.values]

summary_mlp.to_excel('mlp_sekwencyjnie_nasze_dane_zagregowane.xlsx', index=False)

In [ ]:
loss_df = pd.DataFrame({
    "epoch": np.arange(len(loss_history)),
    "loss": loss_history
})

loss_df.to_excel("loss_history_klasyfikacja_nasze_dane.xlsx", index=False)

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

base_params = {
    'hidden_layer_sizes': [32, 16, 8],
    'optimizer': 'sgd',
    'momentum': 0.9,
    'learning_rate': 0.1,
    'epochs': 500
}

hidden_layer_sizes = [
        [128, 64, 32, 16], [64, 32, 16, 8], [32, 16, 8, 4],
        [64, 32, 16], [32, 16, 8], [16, 8, 4],
        [64, 32], [32, 16], [16, 8], [8, 4],
        [64], [32], [16], [8]
]
learning_rates = [0.001, 0.01, 0.05, 0.1]
epochs_list = [1500, 1200, 900, 500]
optimizers = ["sgd", "adam"]
momentum_values = [0.6, 0.7, 0.8, 0.9]
repeat = 5


results = []

def train_and_evaluate(params, r):
    solver = params['optimizer']  # 'sgd' lub 'adam'
    mom = params['momentum'] if solver == 'sgd' else 0.0
    
    mlp = MLPClassifier(
        hidden_layer_sizes=tuple(params['hidden_layer_sizes']),
        activation='relu',
        solver=solver,
        learning_rate_init=params['learning_rate'],
        momentum=mom,
        max_iter=params['epochs'],
        random_state=r
    )
    
    mlp.fit(X_train, y_train_lbl)
    
    y_pred_train = mlp.predict(X_train)
    y_pred_test = mlp.predict(X_test)
    
    results.append({
        'hidden_layers': str(params['hidden_layer_sizes']),
        'optimizer': solver,
        'momentum': mom,
        'learning_rate': params['learning_rate'],
        'epochs': params['epochs'],
        'repeat': r,
        'acc_train': accuracy_score(y_train_lbl, y_pred_train),
        'precision_train': precision_score(y_train_lbl, y_pred_train, average='macro', zero_division=0),
        'recall_train': recall_score(y_train_lbl, y_pred_train, average='macro', zero_division=0),
        'f1_train': f1_score(y_train_lbl, y_pred_train, average='macro', zero_division=0),
        'acc_test': accuracy_score(y_test_lbl, y_pred_test),
        'precision_test': precision_score(y_test_lbl, y_pred_test, average='macro', zero_division=0),
        'recall_test': recall_score(y_test_lbl, y_pred_test, average='macro', zero_division=0),
        'f1_test': f1_score(y_test_lbl, y_pred_test, average='macro', zero_division=0)
    })

# Hidden layers
for h in hidden_layer_configs:
    params = base_params.copy()
    params['hidden_layer_sizes'] = h
    for r in range(1, repeat+1):
        train_and_evaluate(params, r)

# Learning rate
for lr in learning_rates:
    params = base_params.copy()
    params['learning_rate'] = lr
    for r in range(1, repeat+1):
        train_and_evaluate(params, r)

# Optimizer
for opt in optimizers:
    params = base_params.copy()
    params['optimizer'] = opt
    for r in range(1, repeat+1):
        train_and_evaluate(params, r)

# Momentum (tylko dla SGD)
for mom in momentum_values:
    params = base_params.copy()
    params['momentum'] = mom
    for r in range(1, repeat+1):
        train_and_evaluate(params, r)

# Epochs
for ep in epochs_list:
    params = base_params.copy()
    params['epochs'] = ep
    for r in range(1, repeat+1):
        train_and_evaluate(params, r)

df_results = pd.DataFrame(results)
df_results.to_excel('mlp_biblioteka_sekwencyjnie_nasze_dane.xlsx', index=False)

In [27]:
# Zagregowane wyniki (średnia po powtórzeniach)
summary_mlp = df_mlp.groupby(['hidden_layers', 'optimizer', 'momentum', 'learning_rate', 'epochs']) \
    .agg({
        'acc_train': ['mean', 'min', 'max'],
        'precision_train': ['mean', 'min', 'max'],
        'recall_train': ['mean', 'min', 'max'],
        'f1_train': ['mean', 'min', 'max'],
        'acc_test': ['mean', 'min', 'max'],
        'precision_test': ['mean', 'min', 'max'],
        'recall_test': ['mean', 'min', 'max'],
        'f1_test': ['mean', 'min', 'max']
    }).reset_index()

summary_mlp.columns = ['_'.join(col).strip('_') for col in summary_mlp.columns.values]

df_results.to_excel('mlp_biblioteka_sekwencyjnie_nasze_dane_zagregowane.xlsx', index=False)